In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
from sklearn.decomposition import PCA
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html



In [ ]:
columns = ['count', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'dayofweek','hour']

cols_normalize = ['temp','atemp','humidity','windspeed']

In [ ]:
df = pd.read_csv('../input/bike-sharing-demand/train.csv', parse_dates=['datetime'])
df_test = pd.read_csv('../input//bike-sharing-demand/test.csv', parse_dates=['datetime'])

In [ ]:
# We need to convert datetime to numeric for training.
# Let's extract key features into separate numeric columns
def add_features(df):
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['hour'] = df['datetime'].dt.hour

In [ ]:
add_features(df)
add_features(df_test)

In [ ]:
df["count"] = df["count"].map(np.log1p)

In [ ]:
df[cols_normalize].head()

In [ ]:
# Normalize the dataset
scaler = StandardScaler()

In [ ]:
scaler.fit(df[cols_normalize])

In [ ]:
def transform_data(scaler, df, columns):
    transformed_data = scaler.transform(df[columns])
    df_transformed = pd.DataFrame(transformed_data, columns=columns)
    
    for col in df_transformed.columns:
        df[col] = df_transformed[col]

In [ ]:
transform_data(scaler, df, cols_normalize)
transform_data(scaler, df_test, cols_normalize)

Data prep work is finished. Now let's do the PCA.

In [ ]:
# We are not going to use numeric features: 'temp','atemp','humidity','windspeed'
# Instead, we are going to use new components (aka features) generated by PCA for model training and testing
selected_columns = ['count', 'season', 'holiday', 'workingday', 'weather','year', 'month', 'day', 'dayofweek','hour']

# PCA Training
selected_colums_for_pca = ['temp','atemp','humidity','windspeed']

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Initalize PCA. With n_components=.9 means we are retaining 90% of info.
pca = PCA(n_components=.9)

In [ ]:
# Fit PCA
pca.fit(df[selected_colums_for_pca])

In [ ]:
# Transform PCA function
def pca_transform(pca,df,columns):
    transformed_data = pca.transform(df[columns])
    
    tcols = []
    for cols in range(pca.n_components_):
        tcols.append("component_"+str(cols))
        
    df_tranformed = pd.DataFrame(transformed_data,columns=tcols)
        
    for column in df_tranformed.columns:
        df[column] = df_tranformed[column]
        
    df.drop(columns,inplace=True,axis=1)
    return tcols

In [ ]:
# PCA on train
new_cols = pca_transform(pca,df,selected_colums_for_pca)

In [ ]:
# PCA on test
pca_transform(pca,df_test,selected_colums_for_pca)

In [ ]:
for col in new_cols:
    selected_columns.append(col)

In [ ]:
# Shuffle data
np.random.seed(5)
l = list(df.index)
np.random.shuffle(l)
df = df.iloc[l]

In [ ]:
# Split data
rows = df.shape[0]
train = int(.7 * rows)
test = int(.3 * rows)

In [ ]:
df_pca = df[:train]
df_validation_pca = df[train:]
df_test_pca = df_test

## We have completed the PCA. Now let's use XGBoost for prediction

In [ ]:
!conda install -y -c conda-forge xgboost

In [ ]:
import xgboost as xgb

In [ ]:
df_train_pca = df_pca
df_validation_pca = df_validation_pca

In [ ]:
y_train = df_train_pca.iloc[:,:1]
x_train = df_train_pca.iloc[:,1:]

In [ ]:
y_validation = df_validation_pca.iloc[:,:1]
x_validation = df_validation_pca.iloc[:,1:]

In [ ]:
# Initalize regressor
regressor = xgb.XGBRegressor(max_depth=5,n_estimators=150, num_round=150,eta=0.1,subsample=0.7)

In [ ]:
# Fit regressor
regressor.fit(x_train,y_train,eval_set=[(x_train,y_train),(x_validation,y_validation)])

In [ ]:
# Evaluate results
evals_results = regressor.evals_result()
evals_results["validation_0"]["rmse"]

In [ ]:
training_rounds = range(len(evals_results["validation_0"]["rmse"]))

In [ ]:
# Draw scatter plot of train and validation result rmse
plt.scatter(training_rounds,evals_results["validation_0"]["rmse"],label='training loss')
plt.scatter(training_rounds,evals_results["validation_1"]["rmse"],label='validation loss')
plt.grid(True)
plt.xlabel("RMSE")
plt.xlabel("Iteration")
plt.show()

In [ ]:
# check feature importance 
xgb.plot_importance(regressor)

In [ ]:
# Prepare data to submission to Kaggel 
df_test_pca = pd.read_csv("bike_test_pca.csv",parse_dates=['datetime'])

In [ ]:
x_df_test = df_test_pca.iloc[:,1:]

In [ ]:
preds = regressor.predict(x_df_test)

In [ ]:
df_test_pca['count'] = np.expm1(preds)

In [ ]:
df_test_pca[['datetime','count']].to_csv('../output/predicted_count_pca.csv',index=False)